In [7]:
import numpy as np
import os
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import pip
import sys
import cv2

In [8]:
import numpy as np
import os
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import cv2
import pickle

frontFace_cascade = cv2.CascadeClassifier('XML/haarcascade_frontalface_alt.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()

BaseDir   = os.path.dirname(os.path.abspath('__file__'))
imagesDir = os.path.join(BaseDir, 'images') 
print (imagesDir)

current_id = 0
label_id = {}
y_labels = []
x_train  = []


 
for root, dirs, files in os.walk(imagesDir):
    for filee in files:
        if filee.endswith('png') or filee.endswith('JPG') or filee.endswith('jpeg'):
            path  = os.path.join(root, filee)
            label = os.path.basename(root).replace(" ","-").lower() 
            #print label
            pill_image = Image.open(path).convert('L') #convert to GrayScale
            image_array = np.array(pill_image, 'uint8')
            if not label in label_id:
                label_id[label] = current_id
                current_id+=1
            id_=label_id[label]
            faces = frontFace_cascade.detectMultiScale(image_array, 1.2, 5)
            for (x, y, w, h) in faces:
                roi = image_array[y:y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id_)
# print (y_labels)
with open('label.pickle', 'wb') as f: # f mean file # wb mean write bytes
    pickle.dump(label_id, f)
    
recognizer.train(x_train, np.array(y_labels))
recognizer.save("trainner2.yml")

# img = imread('johncena.jpg')
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#     if cv2.waitkey(20) & 0xff == ord('q'):
#         break
# cv2.destroyAllWindows()


C:\Users\abdo\Desktop\Anconda\images


In [ ]:
import pickle 
frontFace_cascade = cv2.CascadeClassifier('XML/haarcascade_frontalface_alt.xml') 
recognizer = cv2.face.LBPHFaceRecognizer_create() 
recognizer.read("trainner2.yml")

labels = {"person_name":1}

size = 160, 160
rename = 'bad_img_'

with open('label.pickle', 'rb') as f: # rb mean read bytes 
    og_labels = pickle.load(f)
    labels = {v:k for k,v in og_labels.items()} ## v:k mean value\key

BaseDir = os.path.dirname(os.path.abspath('__file__'))
afterSiz = os.path.join(BaseDir, 'sized.thumbnail')
allImages = os.path.join(BaseDir, 'test_images')
i=0
for root, dirs, files in os.walk(allImages):
    print(root)
    for filee in files:
        path = os.path.join(root, filee)
        img  = cv2.imread(path)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#         gray = Image.open(path).convert('L') 
        faces = frontFace_cascade.detectMultiScale(gray, 1.2, 5)
        for x, y, w, h in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_frame = img[y:y+h, x:x+w]
            id_, conf = recognizer.predict(gray)
            if(conf >= 55):
                print(id_)
                name = labels[id_]
                color = (255, 255, 255)
                stroke = 2
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(img, name, (x,y), font, 1, color, stroke,cv2.LINE_AA) ##font-size : 1
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.imshow('img', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# faces = frontFace_cascade.detectMultiScale(gray, 1.3, 5)
# for x, y, w, h in faces:
#     roi_gray = gray[y:y+h, x:x+w]
#     roi_frame = img[y:y+h, x:x+w]
#     id_, conf = recognizer.predict(gray)
#     print (id_)
#     print (conf)
#     if(conf >= 45):
#         print(id_)
#         name = labels[id_]
#         color = (255, 255, 255)
#         stroke = 2
#         font = cv2.FONT_HERSHEY_SIMPLEX
#         cv2.putText(img, name, (x,y), font, 1, color, stroke,cv2.LINE_AA) ##font-size : 1
#     else:
#         print("no")

# cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)    
# cv2.imshow('img', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows() 

C:\Users\abdo\Desktop\Anconda\test_images
1
